In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
#https://github.com/fraunhoferhhi/BerlinV2X
sidelink='/data22/bde/Data/BerlinV2X/sidelink_dataframe.parquet'
cellular='/data22/bde/Data/BerlinV2X/cellular_dataframe.parquet'
df = pd.read_parquet(cellular)
df.info(verbose=True)
df.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 207434 entries, 2021-06-22 09:49:00+02:00 to 2021-06-24 18:59:59+02:00
Data columns (total 159 columns):
 #    Column                        Dtype         
---   ------                        -----         
 0    device                        object        
 1    ping_ms                       float64       
 2    datarate                      float64       
 3    jitter                        float64       
 4    ts_gps                        datetime64[ns]
 5    Latitude                      float64       
 6    Longitude                     float64       
 7    Altitude                      float64       
 8    speed_kmh                     float64       
 9    COG                           float64       
 10   precipIntensity               float64       
 11   precipProbability             float64       
 12   temperature                   float64       
 13   apparentTemperature           float64       
 14   dewPoint            

,device,ping_ms,datarate,jitter,ts_gps,Latitude,Longitude,Altitude,speed_kmh,COG,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,cloudCover,uvIndex,visibility,Traffic Jam Factor,Traffic Street Name,Traffic Distance,Pos in Ref Round,measurement,area,PCell_RSRP_1,PCell_RSRP_2,PCell_RSRP_max,PCell_RSRQ_1,PCell_RSRQ_2,PCell_RSRQ_max,PCell_RSSI_1,PCell_RSSI_2,PCell_RSSI_max,PCell_SNR_1,PCell_SNR_2,PCell_E-ARFCN,PCell_Downlink_Num_RBs,PCell_Downlink_TB_Size,PCell_Downlink_RBs_MCS_0,PCell_Downlink_RBs_MCS_1,PCell_Downlink_RBs_MCS_2,PCell_Downlink_RBs_MCS_3,PCell_Downlink_RBs_MCS_4,PCell_Downlink_RBs_MCS_5,PCell_Downlink_RBs_MCS_6,PCell_Downlink_RBs_MCS_7,PCell_Downlink_RBs_MCS_8,PCell_Downlink_RBs_MCS_9,PCell_Downlink_RBs_MCS_10,PCell_Downlink_RBs_MCS_11,PCell_Downlink_RBs_MCS_12,PCell_Downlink_RBs_MCS_13,PCell_Downlink_RBs_MCS_14,PCell_Downlink_RBs_MCS_15,PCell_Downlink_RBs_MCS_16,PCell_Downlink_RBs_MCS_17,PCell_Downlink_RBs_MCS_18,PCell_Downlink_RBs_MCS_19,PCell_Downlink_RBs_MCS_20,PCell_Downlink_RBs_MCS_21,PCell_Downlink_RBs_MCS_22,PCell_Downlink_RBs_MCS_23,PCell_Downlink_RBs_MCS_24,PCell_Downlink_RBs_MCS_25,PCell_Downlink_RBs_MCS_26,PCell_Downlink_RBs_MCS_27,PCell_Downlink_RBs_MCS_28,PCell_Downlink_RBs_MCS_29,PCell_Downlink_RBs_MCS_30,PCell_Downlink_RBs_MCS_31,PCell_Downlink_Average_MCS,PCell_Uplink_Num_RBs,PCell_Uplink_TB_Size,PCell_Uplink_Tx_Power_(dBm),PCell_Cell_ID,PCell_Downlink_frequency,PCell_Uplink_frequency,PCell_Downlink_bandwidth_MHz,PCell_Uplink_bandwidth_MHz,PCell_Cell_Identity,PCell_TAC,PCell_Band_Indicator,PCell_MCC,PCell_MNC_Digit,PCell_MNC,PCell_Allowed_Access,PCell_freq_MHz,SCell_RSRP_1,SCell_RSRP_2,SCell_RSRP_max,SCell_RSRQ_1,SCell_RSRQ_2,SCell_RSRQ_max,SCell_RSSI_1,SCell_RSSI_2,SCell_RSSI_max,SCell_SNR_1,SCell_SNR_2,SCell_E-ARFCN,SCell_Downlink_Num_RBs,SCell_Downlink_TB_Size,SCell_Downlink_RBs_MCS_0,SCell_Downlink_RBs_MCS_1,SCell_Downlink_RBs_MCS_2,SCell_Downlink_RBs_MCS_3,SCell_Downlink_RBs_MCS_4,SCell_Downlink_RBs_MCS_5,SCell_Downlink_RBs_MCS_6,SCell_Downlink_RBs_MCS_7,SCell_Downlink_RBs_MCS_8,SCell_Downlink_RBs_MCS_9,SCell_Downlink_RBs_MCS_10,SCell_Downlink_RBs_MCS_11,SCell_Downlink_RBs_MCS_12,SCell_Downlink_RBs_MCS_13,SCell_Downlink_RBs_MCS_14,SCell_Downlink_RBs_MCS_15,SCell_Downlink_RBs_MCS_16,SCell_Downlink_RBs_MCS_17,SCell_Downlink_RBs_MCS_18,SCell_Downlink_RBs_MCS_19,SCell_Downlink_RBs_MCS_20,SCell_Downlink_RBs_MCS_21,SCell_Downlink_RBs_MCS_22,SCell_Downlink_RBs_MCS_23,SCell_Downlink_RBs_MCS_24,SCell_Downlink_RBs_MCS_25,SCell_Downlink_RBs_MCS_26,SCell_Downlink_RBs_MCS_27,SCell_Downlink_RBs_MCS_28,SCell_Downlink_RBs_MCS_29,SCell_Downlink_RBs_MCS_30,SCell_Downlink_RBs_MCS_31,SCell_Downlink_Average_MCS,SCell_Uplink_Num_RBs,SCell_Uplink_TB_Size,SCell_Uplink_Tx_Power_(dBm),SCell_Cell_ID,SCell_Downlink_frequency,SCell_Uplink_frequency,SCell_Downlink_bandwidth_MHz,SCell_Uplink_bandwidth_MHz,SCell_Cell_Identity,SCell_TAC,SCell_Band_Indicator,SCell_MCC,SCell_MNC_Digit,SCell_MNC,SCell_Allowed_Access,SCell_freq_MHz,scenario,drive_mode,target_datarate,direction,measured_qos,operator
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-06-22 09:49:00+02:00,pc4,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,0,None,-83.476250,-85.292500,-83.476250,-14.799375,-14.771875,-14.573125,-48.676250,-50.520625,-50.520625,23.591,21.963,1300.0,39870.0,6433701.0,NaN,156.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,78.0,453.0,1452.5,4480.0,6442.5,12867.5,10880.0,NaN,NaN,3031.5,27.0,973.0,28004.0,120.854749,246.0,1300.0,19300.0,20,20,33802242.0,1494.0,3.0,262.0,2.0,1.0,0.0,1800.0,-84.980625,-86.231875,-84.930000,-10.736250,-9.635625,-9.614375,-54.243750,-56.594375,-56.594375,25.171,24.226,3050.0,46504.0,8422381.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
df['measured_qos'].unique()
df_datarate = df[df['measured_qos'] == 'datarate']
#df_datarate.info(verbose=True)
df_delay = df[df['measured_qos'] == 'delay']
#df_delay.info(verbose=True)
df_delay.head()

In [ ]:
df_downlink = df[df['direction'] == 'downlink']
#df_downlink.info(verbose=True)
df_uplink = df[df['direction'] == 'uplink']
#df_uplink.info(verbose=True)

In [ ]:
df_pc1 = df[df['device'] == 'pc1']
df_pc2 = df[df['device'] == 'pc2']
df_pc3 = df[df['device'] == 'pc3']
df_pc4 = df[df['device'] == 'pc4']
#df_pc?.info()
selected_df = df_pc1

In [ ]:
#df = df.drop(columns=['device', 'ping_ms'])
df_new = selected_df[['datarate', 'PCell_RSRP_1','PCell_RSRP_2','PCell_RSRQ_1','PCell_RSRQ_2','PCell_RSSI_1','PCell_RSSI_2','PCell_SNR_1','PCell_SNR_2', 'operator']]
df_new.dropna(inplace=True)
df_new.info()

In [ ]:
#df_new.head(100)
df_new.to_csv(f'data/BERLINV2X/test.csv', encoding='utf-8', index=False)
#df_new.describe()

In [ ]:
df_new['datetime'] = df_new.index

groups=(df_new['datetime'].diff()>pd.Timedelta(seconds=1)).cumsum()+1
max_len = 0
continous_groups = []
for i , group in df_new.groupby(groups):
    if len(group) > 4096:
        max_len = len(group)
        continous_groups.append(group)
        print(len(group))
for group in continous_groups:
    group.info()

In [ ]:
longest = continous_groups[0]
longest.head()

In [ ]:
#df.plot(figsize=(30,20))

In [ ]:
df = df[['datarate', 'PCell_RSRP_1', 'PCell_RSRP_2', 'PCell_RSRQ_1', 'PCell_RSRQ_2', 'PCell_RSSI_1', 'PCell_RSSI_2', 'PCell_SNR_1', 'PCell_SNR_2', 'PCell_Cell_ID', 'PCell_Cell_Identity', 'SCell_Cell_ID', 'SCell_Cell_Identity', 'operator']]
df.info()

In [ ]:
df.head(100)

In [ ]:
df.info(verbose=True)

In [ ]:
df.head()